# import

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from category_encoders import *
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import scipy
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
import pickle
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import confusion_matrix
import IPython.display as ipd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
import pandas as pd
import numpy as np
import copy
from scipy.special import softmax
from torchcontrib.optim import SWA

# sort out data

## 读取标签

In [3]:
root_dir="../var/data/"
df_train_user = pd.read_csv(root_dir+'train_semi_final/user.csv')
pre_df_train_user = pd.read_csv(root_dir+'train_preliminary/user.csv')
df_label = pd.concat([pre_df_train_user, df_train_user])

## 读取路径

In [9]:
import os
paths = []
names = []
target_dir = '../var/model_ret_dicts/hyr/'
for filename in os.listdir(target_dir):
    if(filename[0] == '.'):
        continue
    paths.append(target_dir + filename)
    names.append(filename)
    
target_dir = '../var/model_ret_dicts/fjw/'
for filename in os.listdir(target_dir):
    if(filename[0] == '.') :
        continue
    paths.append(target_dir + filename)
    names.append(filename)
names = list(map(lambda x : x.replace('model_', '') ,names))

## 读取数据

In [10]:
model_ret_dicts = []
for p in paths:
    ret = pickle.load(open(p, 'rb'))
    model_ret_dicts.append(ret)

## 单模型效果检验

In [11]:
diff_gender = np.zeros((len(model_ret_dicts), len(model_ret_dicts))).astype('int')
diff_age = np.zeros((len(model_ret_dicts), len(model_ret_dicts))).astype('int')

for i in range(len(model_ret_dicts)):
    for j in range(len(model_ret_dicts)):
        if i == j:
            continue
        if diff_gender[j][i] != 0:
            diff_gender[i][j] = diff_gender[j][i]
            diff_age[i][j] = diff_age[j][i]
            continue
        gender_val = model_ret_dicts[i]['test_gender']
        age_val = model_ret_dicts[i]['test_age']
        gender_pre_i = np.argmax(gender_val, axis = 1)+ 1
        age_pre_i = np.argmax(age_val, axis = 1)+ 1
        
        gender_val = model_ret_dicts[j]['test_gender']
        age_val = model_ret_dicts[j]['test_age']
        gender_pre_j = np.argmax(gender_val, axis = 1)+ 1
        age_pre_j = np.argmax(age_val, axis = 1)+ 1
        
        diff_gender[i][j] = (gender_pre_i != gender_pre_j).sum()
        diff_age[i][j] = (age_pre_i != age_pre_j).sum()


In [12]:
df_age_diff = pd.DataFrame(diff_age, columns=names, index=names)
df_gender_diff = pd.DataFrame(diff_gender, columns=names, index=names)

In [15]:
def get_df_info(name, target, pre):
    right_num = []
    c_m = confusion_matrix(target,  pre)
    n = pd.DataFrame(c_m).sum(axis= 1)
    for i, _ in enumerate(c_m):
        right_num.append(c_m[i][i])
    df_info = pd.DataFrame()
    df_info[name] =  np.array(right_num) / n
    return df_info

df_infos = []
for model_idx in range(len(model_ret_dicts)):
    gender_val = model_ret_dicts[model_idx]['train_gender']
    age_val = model_ret_dicts[model_idx]['train_age']
    gender_pre = np.argmax(gender_val, axis = 1)+ 1
    age_pre = np.argmax(age_val, axis = 1)+ 1
    acc_gender = accuracy_score(df_label.gender.values, gender_pre) 
    acc_age = accuracy_score(df_label.age.values, age_pre) 
    logging.info("%s : %f, %f, %f" % (names[model_idx], acc_gender, acc_age, acc_gender + acc_age)) 
    df_info = get_df_info(names[model_idx], df_label.age.values, age_pre)
    df_infos.append(df_info)
pd.concat(df_infos, axis = 1)

2020-07-23 14:26:12,793 - INFO - one_50dim_wv : 0.690000, 0.143333, 0.833333
2020-07-23 14:26:12,850 - INFO - one_50dim_glove_origin_lr0003_wd0_dp01_len256 : 0.696667, 0.173333, 0.870000
2020-07-23 14:26:12,855 - INFO - pair_200acc_glove_lr0003_wd0_dp04 : 0.710000, 0.203333, 0.913333
2020-07-23 14:26:12,859 - INFO - re2_pair_50dim_wv : 0.696667, 0.156667, 0.853333
2020-07-23 14:26:12,864 - INFO - pair_50dim_glove_smooth_lr0003 : 0.623333, 0.133333, 0.756667
2020-07-23 14:26:12,869 - INFO - one_50dim_origin100glove_lr0003_wd0_dp03 : 0.710000, 0.180000, 0.890000
2020-07-23 14:26:12,873 - INFO - pair_200acc_wv_lr0003_label_smooth : 0.623333, 0.143333, 0.766667
2020-07-23 14:26:12,877 - INFO - one_aa100_wv_glove_lr0003_wd0_dp03 : 0.690000, 0.223333, 0.913333
2020-07-23 14:26:12,881 - INFO - one_50dim_glove_lr0002_wd0_dp03_len64 : 0.666667, 0.156667, 0.823333
2020-07-23 14:26:12,886 - INFO - one_200acc_wv_wd0_dp03 : 0.693333, 0.183333, 0.876667
2020-07-23 14:26:12,890 - INFO - one_200acc_wv

,one_50dim_wv,one_50dim_glove_origin_lr0003_wd0_dp01_len256,pair_200acc_glove_lr0003_wd0_dp04,re2_pair_50dim_wv,pair_50dim_glove_smooth_lr0003,one_50dim_origin100glove_lr0003_wd0_dp03,pair_200acc_wv_lr0003_label_smooth,one_aa100_wv_glove_lr0003_wd0_dp03,one_50dim_glove_lr0002_wd0_dp03_len64,one_200acc_wv_wd0_dp03,...,ESIM_glove50.pk,ESIM_wv50_0.003_shuffle_length100_reverse.pk,ESIM_wv200.pk,ESIM_Single_wv50_0.003_shuffle_length100_reverse.pk,ESIM_Single_glove50.pk,ESIM_glove200.pk,ESIM_Single_wv50_0.3_shuffle_length100_reverse.pk,ESIM_Single_glove50_0.003_shuffle_length88_reverse.pk,transformer_glove50_shuffle.pk,ESIM_Single_glove200.pk
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.117647,0.000000,0.000000,0.176471,0.764706,0.058824,0.000000,0.117647,0.000000,0.176471
1,0.363636,0.163636,0.181818,0.436364,0.290909,0.018182,0.363636,0.290909,0.072727,0.290909,...,0.490909,0.600000,0.709091,0.763636,0.090909,0.345455,0.509091,0.418182,0.472727,0.545455
2,0.134615,0.307692,0.442308,0.153846,0.230769,0.000000,0.134615,0.442308,0.403846,0.250000,...,0.173077,0.307692,0.173077,0.019231,0.038462,0.115385,0.230769,0.096154,0.076923,0.000000
3,0.222222,0.333333,0.388889,0.240741,0.203704,0.611111,0.259259,0.518519,0.277778,0.277778,...,0.092593,0.111111,0.148148,0.000000,0.018519,0.148148,0.129630,0.185185,0.166667,0.129630
4,0.052632,0.078947,0.131579,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,...,0.078947,0.052632,0.052632,0.000000,0.000000,0.052632,0.078947,0.026316,0.078947,0.000000
5,0.048780,0.146341,0.048780,0.048780,0.024390,0.487805,0.048780,0.000000,0.121951,0.073171,...,0.195122,0.097561,0.048780,0.073171,0.024390,0.195122,0.195122,0.146341,0.317073,0.146341
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.280000,...,0.200000,0.040000,0.000000,0.000000,0.120000,0.240000,0.040000,0.280000,0.280000,0.240000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.100000,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.200000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000


## 构造stack特征

In [13]:
train_x = np.zeros((3000000, 12 * len(model_ret_dicts))).astype('float32')
test_x = np.zeros((1000000, 12 * len(model_ret_dicts))).astype('float32')
for i in range(len(model_ret_dicts)):
    train_x[:, i*12:i*12+2] = model_ret_dicts[i]['train_gender']
    train_x[:, i*12+2:i*12+12] = model_ret_dicts[i]['train_age']
                  
    test_x[:, i*12:i*12+2] = model_ret_dicts[i]['test_gender']                  
    test_x[:, i*12+2:i*12+12] = model_ret_dicts[i]['test_age']                  

In [15]:
class_20_val_np = np.load('../var/model_ret_dicts/class20_val.npy')
class_20_test_np = np.load('../var/model_ret_dicts/class20_test.npy')
train_x = np.concatenate([train_x, class_20_val_np], axis=1)
test_x = np.concatenate([test_x, class_20_test_np], axis=1)
train_x.shape, test_x.shape

((3000000, 404), (1000000, 404))

In [16]:
np.save("../var/test_x.npy",test_x)

# stack results

## NN

In [18]:
from collections import namedtuple
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from transformers import *
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm


2020-07-23 12:49:58,742 - INFO - PyTorch version 1.1.0 available.
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

### 调参

In [19]:

ARG = namedtuple('ARG', [
    'batch_size',
    'epoch',
    'lr',
    'weight_decay',
    'n_worker',
    'device',
    'n_fold'
])
 
args = ARG(
    batch_size = 1024,
    epoch = 10,
    lr = 0.005,
    weight_decay = 0.1,
    n_worker = 0,
    n_fold = 5,
    device=torch.device("cuda:3"),
#     device=torch.device("cpu"),

)

### model

In [20]:
class GeLU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1. + torch.tanh(x * 0.7978845608 * (1. + 0.044715 * x * x)))

class Dense(nn.Module):
    def __init__(self):
        super().__init__()
        
        in_feature = 12 * len(model_ret_dicts)+20
        
        hidden = 324
        out_feature = 256
        self.dense = nn.Sequential(
            nn.Linear(in_feature, hidden),
            nn.Tanh(),
            nn.Linear(hidden, out_feature),
        )
        self.decode_gender = nn.Linear(out_feature, 2)
        self.decode_age = nn.Linear(out_feature, 10)
   
    def forward(self, x, gender = None, age = None):
        
        hidden = self.dense(x)
        output_gender = self.decode_gender(hidden)
        output_age = self.decode_age(hidden)
        
        if gender is None:
            return output_gender, output_age
        
        ce = nn.CrossEntropyLoss()
        loss_gender = ce(output_gender, gender.long())
        loss_age = ce(output_age, age.long())
        loss = loss_gender + loss_age
        return loss, loss_gender, loss_age, output_gender, output_age

### trainer

In [21]:
def swa(logger, model, model_dir, model_path_list, swa_start):
    """
    :param logger: ...
    :param model: ...
    :param model_dir: ...
    :param model_path_list: this model path list should be increased by steps
    :param swa_start: the epoch when averaging begins. (start with 0)
    :return: model path list extend with swa model
    """

    assert 1 < swa_start <= len(model_path_list) - 1, \
        f'Using swa, swa start should smaller than {len(model_path_list) - 1} and bigger than 1'

    swa_model = copy.deepcopy(model)
    swa_n = 0.

    with torch.no_grad():
        for _ckpt in model_path_list[swa_start:]:
            logger.info(f'Load model from {_ckpt}')
            model.load_state_dict(torch.load(os.path.join(model_dir, _ckpt, 'model.pt'),
                                             map_location=torch.device('cpu')))
            tmp_para_dict = dict(model.named_parameters())

            alpha = 1. / (swa_n + 1.)

            for name, para in swa_model.named_parameters():
                para.copy_(tmp_para_dict[name].data.clone() * alpha + para.data.clone() * (1. - alpha))

            swa_n += 1

    swa_model_dir = os.path.join(model_dir, f'checkpoint-swa_start{swa_start}')
    if not os.path.exists(swa_model_dir):
        os.mkdir(swa_model_dir)

    logger.info('Save swa model')

    torch.save(swa_model.state_dict(), os.path.join(swa_model_dir, 'model.pt'))

    model_path_list.append(f'checkpoint-swa_start{swa_start}')

    return model_path_list

In [22]:
output_dir="../model/"
def predict_batch_multi_task(model, train_x, batch_size = args.batch_size):
    len_user_ids = len(train_x)
    pre_list_gender = []
    pre_list_age = []
    
    train_dataset = Data.TensorDataset(torch.tensor(train_x).float())
    data_loader = Data.DataLoader(
        dataset=train_dataset,      
        batch_size=args.batch_size,      
        shuffle=False,
        num_workers = args.n_worker,
    )
    with torch.no_grad():
        for step, data in enumerate(tqdm(data_loader)):
            pre_gender, pre_age = model(data[0].to(args.device))
            pre_list_gender.append(pre_gender.cpu().detach().numpy())
            pre_list_age.append(pre_age.cpu().detach().numpy())      
            
    return {
        'gender' : np.concatenate(pre_list_gender), 
        'age' : np.concatenate(pre_list_age),
    }

def eval_data(model, train_x, gender_labels, age_labels):
    
    ret_dict = predict_batch_multi_task(model, train_x)
    predict_gender = np.argmax(ret_dict['gender'], axis = 1)
    predict_age = np.argmax(ret_dict['age'], axis = 1)
    acc_gender = accuracy_score(gender_labels, predict_gender)
    acc_age = accuracy_score(age_labels, predict_age)
    return acc_gender, acc_age

def train_multi_task(n_fold, train_dataset, val_dataset, test_user_id):
    
    train_x = train_dataset['x']
    train_gender = train_dataset['gender']
    train_age = train_dataset['age']
    logging.info('train number %d, val number %d' % (len(train_x), len(val_dataset['x'])))
    torch_dataset = Data.TensorDataset(torch.tensor(train_x).float(), torch.tensor(train_gender).long(), torch.tensor(train_age).long())
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=True,
        num_workers = args.n_worker,
    )
    
    model = Dense().to(args.device)

#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)
    
    no_decay = ["bias", "gamma","beta"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr = args.lr, weight_decay = args.weight_decay)
#     opt = SWA(optimizer, swa_start=int(len(train_x)//(args.batch_size)) * 1, swa_freq=5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(len(train_x)//(args.batch_size)) * 1, num_training_steps=int(len(train_x) / args.batch_size * args.epoch)
    )

    for epoch in range(args.epoch):
        loss_list, loss_gender_list, loss_age_list = [], [], []
        model.train()
        for step, (batch_train_x, bathc_train_gender, batch_train_age) in enumerate(tqdm(data_loader)):
            
            #forward
            loss, loss_gender, loss_age, pre_gender, pre_age = \
            model(batch_train_x.to(args.device), bathc_train_gender.to(args.device), batch_train_age.to(args.device))
            
            loss_list.append(loss.item())
            loss_gender_list.append(loss_gender.item())
            loss_age_list.append(loss_age.item())
            
            #backward
            optimizer.zero_grad()
#             opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5)
            optimizer.step()
#             opt.step()
            scheduler.step()
        if epoch==args.epoch-1:
            torch.save(model.state_dict(),os.path.join(output_dir,"model_"+str(n_fold+1)+".pt"))
        model.eval()

#         train_acc_gender, train_acc_age = eval_data(model, train_dataset['x'], train_dataset['gender'], train_dataset['age'])
        train_acc_gender,train_acc_age=0,0
        val_acc_gender, val_acc_age = eval_data(model, val_dataset['x'], val_dataset['gender'], val_dataset['age'])
        
        logging.info("flod %d epoch %d : \n loss: %f loss_gender : %f, loss_age : %f, gender : %f, %f, age : %f, %f, score : %f" %\
                     (n_fold, epoch, np.mean(loss_list), np.mean(loss_gender_list), np.mean(loss_age_list), \
                      train_acc_gender, val_acc_gender, train_acc_age, val_acc_age, val_acc_gender + val_acc_age))
        
    val_ret_dict = predict_batch_multi_task(model, val_dataset['x'])
    test_ret_dict = predict_batch_multi_task(model, test_user_id)
    
    return model, val_ret_dict, test_ret_dict

def nn_cross_validation(x_train, gender, age, x_test, func_train, shuffle_fold):
    
    folds = KFold(n_splits=args.n_fold, shuffle=shuffle_fold)
    
    
    x_train_val = np.array(x_train)
    gender_train_val = np.array(gender)
    age_train_val = np.array(age)

    score_gender_val = np.zeros((len(x_train), 2))
    score_age_val = np.zeros((len(x_train), 10))
    
    test_gender = np.zeros((len(x_test), 2))
    test_age = np.zeros((len(x_test), 10))

    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(x_train, gender_train_val)):
        save_trn_idx = trn_idx
        save_val_idx = val_idx
        
        train_x, train_gender, train_age = x_train_val[trn_idx], gender_train_val[trn_idx], age_train_val[trn_idx]
        val_x, val_gender, val_age = x_train_val[val_idx], gender_train_val[val_idx], age_train_val[val_idx]
        train_dataset = {
            'x' : train_x,
            'gender' : train_gender,
            'age' : train_age,
        }
        val_dataset = {
            'x' : val_x,
            'gender' : val_gender,
            'age' : val_age
        }
        
        model, val_ret_dict, test_ret_dict = func_train(n_fold, train_dataset, val_dataset, x_test)
        
        score_gender_val[val_idx] = val_ret_dict['gender']
        score_age_val[val_idx] = val_ret_dict['age']
        
        
        test_gender += softmax(test_ret_dict['gender'], axis=1) / args.n_fold
        test_age += softmax(test_ret_dict['age'], axis=1) / args.n_fold

    return score_gender_val, score_age_val, test_gender, test_age


# debug_number = 2000
# sub_train_x = train_x[:debug_number]
# sub_gender = (df_label.gender.values-1)[:debug_number]
# sub_age = (df_label.age.values-1)[:debug_number]
# sub_test_x = test_x[:debug_number]

sub_train_x = train_x
# sub_gender = (df_label_modify.gender.values-1)
sub_gender = (df_label.gender.values-1)
sub_age = (df_label.age.values-1)
sub_test_x = test_x

score_gender_val, score_age_val, test_gender, test_age = nn_cross_validation(sub_train_x, sub_gender, sub_age, sub_test_x, train_multi_task, False)

2020-07-23 12:50:29,353 - INFO - train number 2400000, val number 600000
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:69: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


2020-07-23 12:51:22,102 - INFO - flod 0 epoch 0 : 
 loss: 1.362071 loss_gender : 0.159777, loss_age : 1.202294, gender : 0.000000, 0.946612, age : 0.000000, 0.521302, score : 1.467913


2020-07-23 12:52:11,114 - INFO - flod 0 epoch 1 : 
 loss: 1.301298 loss_gender : 0.145754, loss_age : 1.155543, gender : 0.000000, 0.950927, age : 0.000000, 0.520672, score : 1.471598


2020-07-23 12:52:58,576 - INFO - flod 0 epoch 2 : 
 loss: 1.294762 loss_gender : 0.144794, loss_age : 1.149968, gender : 0.000000, 0.950952, age : 0.000000, 0.523397, score : 1.474348


...

2020-07-23 13:28:47,068 - INFO - flod 4 epoch 5 : 
 loss: 1.286705 loss_gender : 0.143326, loss_age : 1.143378, gender : 0.000000, 0.950893, age : 0.000000, 0.522970, score : 1.473863


2020-07-23 13:29:37,028 - INFO - flod 4 epoch 6 : 
 loss: 1.284487 loss_gender : 0.143045, loss_age : 1.141442, gender : 0.000000, 0.950907, age : 0.000000, 0.524502, score : 1.475408


2020-07-23 13:30:29,201 - INFO - flod 4 epoch 7 : 
 loss: 1.282045 loss_gender : 0.142613, loss_age : 1.139432, gender : 0.000000, 0.950747, age : 0.000000, 0.526082, score : 1.476828


2020-07-23 13:31:22,209 - INFO - flod 4 epoch 8 : 
 loss: 1.279614 loss_gender : 0.142216, loss_age : 1.137399, gender : 0.000000, 0.950837, age : 0.000000, 0.526173, score : 1.477010


2020-07-23 13:32:14,651 - INFO - flod 4 epoch 9 : 
 loss: 1.276796 loss_gender : 0.141802, loss_age : 1.134994, gender : 0.000000, 0.950910, age : 0.000000, 0.527532, score : 1.478442


In [23]:
acc_gender = accuracy_score(df_label.gender.values, np.argmax(score_gender_val, axis = 1) + 1)
acc_age = accuracy_score(df_label.age.values, np.argmax(score_age_val, axis = 1) + 1)
acc_gender, acc_age, acc_gender + acc_age #33

(0.9511223333333333, 0.5275906666666667, 1.478713)

In [24]:
test_gender_pre = np.argmax(test_gender, axis = 1) + 1
test_age_pre = np.argmax(test_age, axis = 1) + 1
df_submit = pd.DataFrame()
df_submit['user_id'] = list(range(3000001, 4000001))
df_submit['predicted_gender'] = test_gender_pre
df_submit['predicted_age'] = test_age_pre
df_submit.to_csv('submission.csv', index=False)